In [1]:
%run "curve_fit_.ipynb"
%run "data_processing_.ipynb" #HEADER_LEN,X_DIM,Y_DIM,X_left_boundry,X_right_boundry,Y_left_boundry,Y_right_boundry
%run "evaluation_functions_.ipynb"
%run "fitting_functions_.ipynb"
%run "get_temperature_.ipynb"
%run "load_data_.ipynb" # frame rate, shutter delay, and interseted position 
%run "get_backbround_average_.ipynb" 
%run "frame_vs_temperature.ipynb" # get_powers

In [ ]:
folder = ""

In [ ]:
all_files, fivemm_files, velocity_path_list, velocity_list, used_frame_num = get_needed_files(folder)
back_intensity_list = get_background_average(fivemm_files)
power_list = get_powers(all_files)

In [ ]:
Si_melting_W = {
    '103': ,
    '13': ,
    '155': ,
    '190': , 
    '20': ,
    '234': ,
    '30': ,
    '352': ,
    '45': ,
    '68': ,
    '9': 
}

In [ ]:
v_list = ['103', '13','155','190','234', '30', '352', '45', '68']
Dict_1 = dict((k, fivemm_files[k][5:]) for k in v_list) # not included 0W files
back_intensity_list_1 = []
for key in fivemm_files:
    if key in v_list:
        index = list(fivemm_files).index(key)
        back_intensity_list_1.append(back_intensity_list[index])
dRoverR_1 = []
for i, v in enumerate(list(Dict_1.keys())):
    con = []
    for file in Dict_1[v]:
        val = get_peak_dRoverR(file, back_intensity_list_1[i], two_lorentz, two_lorentz)
        con.append(val)
    dRoverR_1.append(con)

In [ ]:
power_used = {'103':[27+2*p for p in range(0,len(dRoverR_1[v_list.index('103')]),1)], 
        '13':[21+2*p for p in range(0,len(dRoverR_1[v_list.index('13')]),1)], 
        '155':[27+2*p for p in range(0,len(dRoverR_1[v_list.index('155')]),1)],
        '190':[29+2*p for p in range(0,len(dRoverR_1[v_list.index('190')]),1)], 
        '20':[23+2*p for p in range(0,len(dRoverR_1[v_list.index('20')]),1)], 
        '234':[29+2*p for p in range(0,len(dRoverR_1[v_list.index('234')]),1)], 
        '30':[25+2*p for p in range(0,len(dRoverR_1[v_list.index('30')]),1)], 
        '352':[31+2*p for p in range(0,len(dRoverR_1[v_list.index('352')]),1)], 
        '45':[25+2*p for p in range(0,len(dRoverR_1[v_list.index('45')]),1)], 
        '68':[25+2*p for p in range(0,len(dRoverR_1[v_list.index('68')]),1)], 
        '9':[21+2*p for p in range(0,len(dRoverR_1[v_list.index('9')]),1)]}

In [ ]:
for i, (key, value) in enumerate(all_files.items()):
    power = [int(x) for x in power_list[i]][1:]
    files = all_files[key]
    
    for p in power:
        dR_list = []
        for file in tqdm(files):
            if "_{}W_".format(p) in str(file):
                print(str(file))
                dR = get_peak_dRoverR(file, back_intensity_list[i], gaus, gaus)
                dR_list.append(dR)
                
        #convert the frame number to position
        #x = [(frame-2.7)*int(key)/92.59 for frame in range(1,len(temp_list)+1)] 
        
        x = [frame for frame in range(0,len(dR_list))] 
        
        
        plt.plot(x, dR_list, marker="o")
        
        #limit temperature to 1500C
        #plt.ylim(0,1500)
        
        #limit position to 1-10mm
        #plt.xlim(0,5)

    plt.savefig("/Users/chiaentsai/Desktop/lab/Temperature_Calibration/0130_dRoverR_vs_frame/{}mmpersec".format(key), format="PNG")
    plt.close()

In [ ]:
for i, v in enumerate(list(Si_melting_W.keys())):
    plt.plot(power_used[v], dRoverR_1[i], '.')
    plt.xlabel("power")
    plt.ylabel("dR/R")
    plt.savefig("/Users/chiaentsai/Desktop/lab/Temperature_Calibration/0130_dRoverR_figure/{}mm.png".format(v), format="PNG")
    plt.close()


### get kappa with Si melt power

In [ ]:
Dict_gold = dict((k, fivemm_files[k][10:]) for k in gold_list)
back_intensity_list_gold = []
for key in fivemm_files:
    if key in gold_list:
        index = list(fivemm_files).index(key)
        back_intensity_list_gold.append(back_intensity_list[index])

#Dict_dRoverR = dict.fromkeys(list(Dict_gold.keys()),[])
dRoverR = []
for i, v in enumerate(list(Dict_gold.keys())):
    con = []
    for file in Dict_gold[v]:
        val = get_peak_dRoverR(file, back_intensity_list_gold[i], two_lorentz, two_lorentz)
        con.append(val)
    dRoverR.append(con)

power_gold = []
for i, (key, value) in enumerate(fivemm_files.items()):
    if key in gold_list:
        power_gold.append(power_list[i][1:])

for i, v in enumerate(list(Dict_gold.keys())):
    plt.plot(power_gold[i][:-1], dRoverR[i][:-1], '.')
    plt.xlabel("power")
    plt.ylabel("dR/R")
    plt.savefig("/Users/chiaentsai/Desktop/lab/Temperature_Calibration/gold_dRoverR_figure/{}mm.png".format(v), format="PNG")
    plt.close()

kappa_gold = []
for i, v in enumerate(list(Dict_gold.keys())):
    power = [float(j) for j in power_gold[i]][:-1]
    y = dRoverR[i][:-1]
    popt, pcov = curve_fit(expo, power, y, p0 = [1, 0.0000005, 1], maxfev = 8000)
    #a, b, c = popt
    plt.plot(power, y, '.')
    y_pred = [expo(j, *popt) for j in power]
    plt.plot(power, y_pred)
    plt.xlabel("power")
    plt.ylabel("dR/R")
    plt.savefig("/Users/chiaentsai/Desktop/lab/Temperature_Calibration/gold_dRoverR_fit_figure/{}mm.png".format(v), format="PNG")
    plt.close()
    f = expo(gold_melting_W[v], *popt)
    gold_m = 1064
    dT = gold_m - 21
    kappa = f / dT
    kappa_gold.append(kappa)
    
Dict_kappa_gold = dict(zip(list(Dict_gold.keys()),kappa_gold))

plt.plot(list(Dict_gold.keys()), kappa_gold, '.')
plt.savefig("/Users/chiaentsai/Desktop/lab/Temperature_Calibration/kappa.png", format="PNG")

In [ ]:
peak_temp = []
for i, v in enumerate(list(Si_melting_W.keys())):
    tem = []
    for r in dRoverR_1[i]:
        t = get_peak_Temp(r, Dict_kappa_gold[v])
#         t = get_peak_Temp(r,avg_kappa)
#         t = get_peak_Temp(r,0.00016498400434141028)
#         t = get_peak_Temp(r, Dict_kappa_gold_avg[v])
#         t = get_peak_Temp(r, avg_kappa_2)
#         t = get_peak_Temp(r, Dict_kappa_avg[v])
        tem.append(t)
    peak_temp.append(tem)
    
Dict_peak_temp = dict(zip(list(Si_melting_W.keys()),peak_temp))